In [75]:
import csv
import math
import sys
from datetime import datetime as dt
from datetime import timezone as tz

import numpy as np
import pandas as pd

import utility

## clean data

### regional

In [198]:
df = pd.read_excel('../GhostPostCC/data/Region_MIDW.xlsx')

In [197]:
df

,Balancing Authority,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),...,Net Generation (MW) from Other Fuel Sources (Imputed),Net Generation (MW) from Coal (Adjusted),Net Generation (MW) from Natural Gas (Adjusted),Net Generation (MW) from Nuclear (Adjusted),Net Generation (MW) from All Petroleum Products (Adjusted),Net Generation (MW) from Hydropower and Pumped Storage (Adjusted),Net Generation (MW) from Solar (Adjusted),Net Generation (MW) from Wind (Adjusted),Net Generation (MW) from Other Fuel Sources (Adjusted),Region
0,AECI,01/01/2022,1,01/01/2022 1:00:00 AM,01/01/2022 7:00:00 AM,"2,235","2,251","1,986",-265,-265,...,NaN,996,299,NaN,NaN,NaN,NaN,691,NaN,MIDW
1,AECI,01/01/2022,2,01/01/2022 2:00:00 AM,01/01/2022 8:00:00 AM,"2,217","2,208","2,039",-169,-169,...,NaN,941,302,NaN,NaN,NaN,NaN,796,NaN,MIDW
2,AECI,01/01/2022,3,01/01/2022 3:00:00 AM,01/01/2022 9:00:00 AM,"2,193","2,204","2,080",-124,-124,...,NaN,951,369,NaN,NaN,NaN,NaN,760,NaN,MIDW
3,AECI,01/01/2022,4,01/01/2022 4:00:00 AM,01/01/2022 10:00:00 AM,"2,255","2,234","2,110",-124,-124,...,NaN,957,386,NaN,NaN,NaN,NaN,767,NaN,MIDW
4,AECI,01/01/2022,5,01/01/2022 5:00:00 AM,01/01/2022 11:00:00 AM,"2,325","2,287","2,138",-149,-149,...,NaN,992,405,NaN,NaN,NaN,NaN,741,NaN,MIDW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273616,YAD,06/30/2022,20,06/30/2022 8:00:00 PM,07/01/2022 12:00:00 AM,NaN,NaN,122,119,119,...,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,NaN,CAR
273617,YAD,06/30/2022,21,06/30/2022 9:00:00 PM,07/01/2022 1:00:00 AM,NaN,NaN,4,2,4,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,CAR
273618,YAD,06/30/2022,22,06/30/2022 10:00:00 PM,07/01/2022 2:00:00 AM,NaN,NaN,0,-1,-1,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,CAR
273619,YAD,06/30/2022,23,06/30/2022 11:00:00 PM,07/01/2022 3:00:00 AM,NaN,NaN,1,-1,-1,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,CAR


In [61]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77303 entries, 0 to 77302
Data columns (total 59 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   Region                                             77303 non-null  object        
 1   UTC time                                           77303 non-null  datetime64[ns]
 2   Local date                                         77303 non-null  datetime64[ns]
 3   Hour                                               77303 non-null  int64         
 4   Local time                                         77303 non-null  datetime64[ns]
 5   Time zone                                          77303 non-null  object        
 6   DF                                                 77303 non-null  int64         
 7   D                                                  77279 non-null  float64       
 8   NG              

In [62]:
df.columns

Index(['Region', 'UTC time', 'Local date', 'Hour', 'Local time', 'Time zone',
       'DF', 'D', 'NG', 'TI', 'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC',
       'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH', 'NG: UNK',
       'Sum (Trade)', 'Sum (Imports)', 'Sum (Exports)', 'CAL', 'CAR', 'CENT',
       'FLA', 'MIDA', 'MIDW', 'NE', 'NW', 'NY', 'SE', 'SW', 'TEN', 'TEX',
       'CAN', 'MEX', 'Balance NG D TI', 'Balance TI Trade', 'Balance NG',
       'CO2 Factor: COL', 'CO2 Factor: NG', 'CO2 Factor: OIL',
       'Positive Gen from COL', 'Positive Gen from NG',
       'Positive Gen from OIL', 'CO2 Emissions: COL', 'CO2 Emissions: NG',
       'CO2 Emissions: OIL', 'CO2 Emissions: Other', 'CO2 Emissions Generated',
       'CO2 Emissions Imported', 'CO2 Emissions Exported',
       'CO2 Emissions Consumed', 'Positive Generation', 'Consumed Electricity',
       'CO2 Emissions Intensity for Generated Electricity',
       'CO2 Emissions Intensity for Consumed Electricity'],
      dtype='obj

In [76]:
cols = ['UTC time', 'Local date', 'Hour', 'Local time',
       'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC', 'NG: SUN',
       'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH']

time_filter = (df['Local time']>'2022-01-01') & (df['Local time']<='2024-01-01')

In [115]:
data = df[time_filter][cols].reset_index(drop=True)
data

,UTC time,Local date,Hour,Local time,Sum (NG),NG: COL,NG: NG,NG: NUC,NG: SUN,NG: OIL,NG: WAT,NG: SUN,NG: WND,NG: OTH
0,2022-01-01 06:00:00,2022-01-01,1,2022-01-01 01:00:00,64543.0,25663.0,12065.0,12192.0,0.0,NaN,569.0,0.0,13584.0,470.0
1,2022-01-01 07:00:00,2022-01-01,2,2022-01-01 02:00:00,63027.0,24282.0,11574.0,12192.0,0.0,NaN,529.0,0.0,13982.0,468.0
2,2022-01-01 08:00:00,2022-01-01,3,2022-01-01 03:00:00,62214.0,23480.0,11296.0,12192.0,0.0,NaN,522.0,0.0,14256.0,468.0
3,2022-01-01 09:00:00,2022-01-01,4,2022-01-01 04:00:00,61693.0,22871.0,11026.0,12198.0,0.0,NaN,529.0,0.0,14600.0,469.0
4,2022-01-01 10:00:00,2022-01-01,5,2022-01-01 05:00:00,61166.0,22423.0,10701.0,12195.0,0.0,NaN,527.0,0.0,14850.0,470.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2024-01-01 01:00:00,2023-12-31,20,2023-12-31 20:00:00,78078.0,29329.7,30487.3,8204.0,2.0,0.0,1088.0,2.0,8596.0,371.0
17516,2024-01-01 02:00:00,2023-12-31,21,2023-12-31 21:00:00,75593.9,27283.2,30073.8,8204.0,2.9,0.0,909.0,2.9,8753.0,368.0
17517,2024-01-01 03:00:00,2023-12-31,22,2023-12-31 22:00:00,74670.1,26415.1,30143.0,8206.0,3.0,0.0,905.0,3.0,8635.0,363.0
17518,2024-01-01 04:00:00,2023-12-31,23,2023-12-31 23:00:00,73296.0,25840.3,30098.7,8203.0,3.0,0.0,878.0,3.0,7908.0,365.0


In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   UTC time    17520 non-null  datetime64[ns]
 1   Local date  17520 non-null  datetime64[ns]
 2   Hour        17520 non-null  int64         
 3   Local time  17520 non-null  datetime64[ns]
 4   Sum (NG)    17520 non-null  float64       
 5   NG: COL     17520 non-null  float64       
 6   NG: NG      17520 non-null  float64       
 7   NG: NUC     17520 non-null  float64       
 8   NG: SUN     17520 non-null  float64       
 9   NG: OIL     5184 non-null   float64       
 10  NG: WAT     17520 non-null  float64       
 11  NG: SUN     17520 non-null  float64       
 12  NG: WND     17520 non-null  float64       
 13  NG: OTH     17520 non-null  float64       
dtypes: datetime64[ns](3), float64(10), int64(1)
memory usage: 1.9 MB


In [117]:
# omit NG: SUN bcs no sun data
# omit NG: OIL bcs too many nans
# change NG: OTH to become (Sum (NG) - ('NG: COL' + 'NG: NG' + 'NG: NUC' + 'NG: WAT' + 'NG: SUN' + 'NG: WND'))

data = data.drop(['NG: SUN', 'NG: OIL'], axis=1)

data['NG: OTH'] = 0
data['NG: OTH'] = data['Sum (NG)'] - data[['NG: COL', 'NG: NG', 'NG: NUC', 'NG: WAT', 'NG: WND']].sum(axis=1)
data = data.drop(['Sum (NG)', 'Local date', 'Hour'], axis=1)

In [118]:
data

,UTC time,Local time,NG: COL,NG: NG,NG: NUC,NG: WAT,NG: WND,NG: OTH
0,2022-01-01 06:00:00,2022-01-01 01:00:00,25663.0,12065.0,12192.0,569.0,13584.0,470.0
1,2022-01-01 07:00:00,2022-01-01 02:00:00,24282.0,11574.0,12192.0,529.0,13982.0,468.0
2,2022-01-01 08:00:00,2022-01-01 03:00:00,23480.0,11296.0,12192.0,522.0,14256.0,468.0
3,2022-01-01 09:00:00,2022-01-01 04:00:00,22871.0,11026.0,12198.0,529.0,14600.0,469.0
4,2022-01-01 10:00:00,2022-01-01 05:00:00,22423.0,10701.0,12195.0,527.0,14850.0,470.0
...,...,...,...,...,...,...,...,...
17515,2024-01-01 01:00:00,2023-12-31 20:00:00,29329.7,30487.3,8204.0,1088.0,8596.0,373.0
17516,2024-01-01 02:00:00,2023-12-31 21:00:00,27283.2,30073.8,8204.0,909.0,8753.0,370.9
17517,2024-01-01 03:00:00,2023-12-31 22:00:00,26415.1,30143.0,8206.0,905.0,8635.0,366.0
17518,2024-01-01 04:00:00,2023-12-31 23:00:00,25840.3,30098.7,8203.0,878.0,7908.0,368.0


In [119]:
data.to_csv('data/electricity_cleaned.csv')

### six months

In [254]:
def combine_data(balancing_auth):
    years = ['2021', '2022', '2023', '2024']
    periods = ['Jan_Jun', 'Jul_Dec']

    dataset = pd.DataFrame()
    for year in years:
        for period in periods:
            try:
                file_path = f"../GhostPostCC/data/EIA930_BALANCE_{year}_{period}.csv"
                df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
                df = df[df['Balancing Authority']==balancing_auth]
                dataset = pd.concat([dataset, df]).reset_index(drop=True)
                print(f'{file_path} combined')
            except FileNotFoundError as e:
                print(e)

    return dataset

In [255]:
dataset = combine_data('MISO')

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: DtypeWarning: Columns (5,6,10,11,13,16,18,19,20,21,22,23,25,26,27,29,32,33,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2021_Jan_Jun.csv combined


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: DtypeWarning: Columns (5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2021_Jul_Dec.csv combined


...

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: DtypeWarning: Columns (5,6,7,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2023_Jul_Dec.csv combined


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: DtypeWarning: Columns (5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2024_Jan_Jun.csv combined
[Errno 2] No such file or directory: '../GhostPostCC/data/EIA930_BALANCE_2024_Jul_Dec.csv'


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2321871258.py:10: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


In [256]:
dataset

,Balancing Authority,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),...,Net Generation (MW) from Other Fuel Sources (Imputed),Net Generation (MW) from Coal (Adjusted),Net Generation (MW) from Natural Gas (Adjusted),Net Generation (MW) from Nuclear (Adjusted),Net Generation (MW) from All Petroleum Products (Adjusted),Net Generation (MW) from Hydropower and Pumped Storage (Adjusted),Net Generation (MW) from Solar (Adjusted),Net Generation (MW) from Wind (Adjusted),Net Generation (MW) from Other Fuel Sources (Adjusted),Region
0,MISO,2021-01-01,1,2021-01-01 01:00:00,2021-01-01 06:00:00,"67,059","67,566","63,940","-4,521","-4,521",...,NaN,"25,367","15,999","11,534",NaN,714,0,"9,486",839,MIDW
1,MISO,2021-01-01,2,2021-01-01 02:00:00,2021-01-01 07:00:00,"65,069","66,270","62,553","-4,435","-4,435",...,NaN,"24,662","16,063","11,544",NaN,664,0,"8,780",840,MIDW
2,MISO,2021-01-01,3,2021-01-01 03:00:00,2021-01-01 08:00:00,"63,767","65,380","61,389","-4,673","-4,673",...,NaN,"25,365","16,057","11,555",NaN,661,0,"6,912",839,MIDW
3,MISO,2021-01-01,4,2021-01-01 04:00:00,2021-01-01 09:00:00,"62,861","64,402","59,712","-5,251","-5,251",...,NaN,"24,811","15,870","11,560",NaN,664,0,"5,967",841,MIDW
4,MISO,2021-01-01,5,2021-01-01 05:00:00,2021-01-01 10:00:00,"62,396","63,640","58,947","-5,408","-5,408",...,NaN,"25,041","15,755","11,565",NaN,668,0,"5,074",844,MIDW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29035,MISO,2024-04-24,20,2024-04-24 20:00:00,2024-04-25 01:00:00,"70,461",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIDW
29036,MISO,2024-04-24,21,2024-04-24 21:00:00,2024-04-25 02:00:00,"70,246",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIDW
29037,MISO,2024-04-24,22,2024-04-24 22:00:00,2024-04-25 03:00:00,"67,846",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIDW
29038,MISO,2024-04-24,23,2024-04-24 23:00:00,2024-04-25 04:00:00,"64,743",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIDW


In [260]:
cols_contain_to_omit = ['Hour Number', 'Data Date', 'Balancing Authority', 'Demand', 'Interchange', 'DIBAs', 'Imputed', 'Adjusted', 'Region']

def filter_columns(df, cols_contain_to_omit):
    for omit in cols_contain_to_omit:
        df = df.loc[:,~df.columns.str.contains(omit, case=False)]
    
    return df

In [261]:
dataset = filter_columns(dataset, cols_contain_to_omit)

In [262]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29040 entries, 0 to 29039
Data columns (total 12 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   Local Time at End of Hour                               29040 non-null  datetime64[ns]
 1   UTC Time at End of Hour                                 29040 non-null  datetime64[ns]
 2   Net Generation (MW)                                     29011 non-null  object        
 3   Net Generation (MW) from Coal                           29011 non-null  object        
 4   Net Generation (MW) from Natural Gas                    29011 non-null  object        
 5   Net Generation (MW) from Nuclear                        29011 non-null  object        
 6   Net Generation (MW) from All Petroleum Products         24 non-null     object        
 7   Net Generation (MW) from Hydropower and Pumped Storage  29

In [263]:
# all nans in the last 25 rows
dataset.tail(30)

,Local Time at End of Hour,UTC Time at End of Hour,Net Generation (MW),Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources
29010,2024-04-23 19:00:00,2024-04-24 00:00:00,"67,906","12,515","22,389","9,549",NaN,"1,809","1,959","19,403",284,NaN
29011,2024-04-23 20:00:00,2024-04-24 01:00:00,"67,450","13,178","24,567","9,553",NaN,"2,320",251,"17,334",248,NaN
29012,2024-04-23 21:00:00,2024-04-24 02:00:00,"66,111","14,894","25,584","9,557",NaN,"1,921",1,"13,903",252,NaN
29013,2024-04-23 22:00:00,2024-04-24 03:00:00,"63,665","14,970","24,634","9,565",NaN,"1,799",0,"12,441",256,NaN
29014,2024-04-23 23:00:00,2024-04-24 04:00:00,"60,306","13,664","23,328","9,567",NaN,"1,169",0,"12,319",259,NaN
29015,2024-04-24 00:00:00,2024-04-24 05:00:00,"58,468","12,639","22,016","9,568",NaN,979,0,"13,005",260,NaN
29016,2024-04-24 01:00:00,2024-04-24 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29017,2024-04-24 02:00:00,2024-04-24 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29018,2024-04-24 03:00:00,2024-04-24 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29019,2024-04-24 04:00:00,2024-04-24 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
dataset[dataset['UTC Time at End of Hour']<'2024-01-01']

,Local Time at End of Hour,UTC Time at End of Hour,Net Generation (MW),Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources
0,2021-01-01 01:00:00,2021-01-01 06:00:00,"63,940","25,367","15,999","11,534",NaN,714,0,"9,486",839,NaN
1,2021-01-01 02:00:00,2021-01-01 07:00:00,"62,553","24,662","16,063","11,544",NaN,664,0,"8,780",840,NaN
2,2021-01-01 03:00:00,2021-01-01 08:00:00,"61,389","25,365","16,057","11,555",NaN,661,0,"6,912",839,NaN
3,2021-01-01 04:00:00,2021-01-01 09:00:00,"59,712","24,811","15,870","11,560",NaN,664,0,"5,967",841,NaN
4,2021-01-01 05:00:00,2021-01-01 10:00:00,"58,947","25,041","15,755","11,565",NaN,668,0,"5,074",844,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
26269,2023-12-31 14:00:00,2023-12-31 19:00:00,"68,586","19,663","26,718","8,205",NaN,811,"1,060","11,799",331,NaN
26270,2023-12-31 15:00:00,2023-12-31 20:00:00,"68,042","19,740","26,912","8,203",NaN,805,976,"11,074",333,NaN
26271,2023-12-31 16:00:00,2023-12-31 21:00:00,"67,827","20,145","26,794","8,204",NaN,802,825,"10,717",340,NaN
26272,2023-12-31 17:00:00,2023-12-31 22:00:00,"68,717","21,403","27,195","8,203",NaN,850,541,"10,181",343,NaN


## process data

In [120]:
source = 'coal'
IN_FILE_NAME = "data/electricity_cleaned.csv"

In [121]:
NUM_FEATURES_DICT = {"coal":6, "nat_gas":6, "nuclear":6, "oil":6, "hydro":11, "solar": 11,
                    "wind":11, "unknown": 6, "biomass": 6, "geothermal":6}

NUM_VAL_DAYS = 30
NUM_TEST_DAYS = 184
TRAINING_WINDOW_HOURS = 24
PREDICTION_WINDOW_HOURS = 24
MODEL_SLIDING_WINDOW_LEN = 24

COAL = 4
NAT_GAS = 5
NUCLEAR = 6
HYDRO = 7
WIND = 8
OTHERS = 9 

FUEL = {3:"coal", 4:"nat_gas", 5:"nuclear", 6:"oil", 7:"hydro", 8:"solar",
                    9:"wind", 10:"unknown"}

SOURCE_TO_SOURCE_COL_MAP = {"coal": COAL, "nat_gas" : NAT_GAS, "nuclear" : NUCLEAR, "hydro" : HYDRO, "wind" : WIND}
SOURCE_COL = SOURCE_TO_SOURCE_COL_MAP[source]
NUM_FEATURES = NUM_FEATURES_DICT[FUEL[SOURCE_COL]]

In [122]:
def initDataset(inFileName, sourceCol):
    dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC time'], index_col=['UTC time'])

    print(dataset.head())
    print(dataset.columns)
    dateTime = dataset.index.values
    
    print("\nAdding features related to date & time...")
    modifiedDataset = utility.addDateTimeFeatures(dataset, dateTime, sourceCol)
    dataset = modifiedDataset
    print("Features related to date & time added")
    
    for i in range(sourceCol, len(dataset.columns.values)):
        col = dataset.columns.values[i]
        dataset[col] = dataset[col].astype(np.float64)
        # print(col, dataset[col].dtype)

    return dataset, dateTime

In [123]:
dataset, dateTime = initDataset(IN_FILE_NAME, SOURCE_COL)

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/1023274769.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC time'], index_col=['UTC time'])


                     Unnamed: 0           Local time  NG: COL   NG: NG  \
UTC time                                                                 
2022-01-01 06:00:00           0  2022-01-01 01:00:00  25663.0  12065.0   
2022-01-01 07:00:00           1  2022-01-01 02:00:00  24282.0  11574.0   
2022-01-01 08:00:00           2  2022-01-01 03:00:00  23480.0  11296.0   
2022-01-01 09:00:00           3  2022-01-01 04:00:00  22871.0  11026.0   
2022-01-01 10:00:00           4  2022-01-01 05:00:00  22423.0  10701.0   

                     NG: NUC  NG: WAT  NG: WND  NG: OTH  
UTC time                                                 
2022-01-01 06:00:00  12192.0    569.0  13584.0    470.0  
2022-01-01 07:00:00  12192.0    529.0  13982.0    468.0  
2022-01-01 08:00:00  12192.0    522.0  14256.0    468.0  
2022-01-01 09:00:00  12198.0    529.0  14600.0    469.0  
2022-01-01 10:00:00  12195.0    527.0  14850.0    470.0  
Index(['Unnamed: 0', 'Local time', 'NG: COL', 'NG: NG', 'NG: NUC', 'NG: WAT

In [127]:
dataset

,Unnamed: 0,Local time,NG: COL,NG: NG,NG: NUC,hour_sin,hour_cos,month_sin,month_cos,weekend,NG: WAT,NG: WND,NG: OTH
UTC time,,,,,,,,,,,,,
2022-01-01 06:00:00,0,2022-01-01 01:00:00,25663.0,12065.0,12192.0,1.000000,6.123234e-17,0.004301,0.999991,1.0,569.0,13584.0,470.0
2022-01-01 07:00:00,1,2022-01-01 02:00:00,24282.0,11574.0,12192.0,0.965926,-2.588190e-01,0.005017,0.999987,1.0,529.0,13982.0,468.0
2022-01-01 08:00:00,2,2022-01-01 03:00:00,23480.0,11296.0,12192.0,0.866025,-5.000000e-01,0.005734,0.999984,1.0,522.0,14256.0,468.0
2022-01-01 09:00:00,3,2022-01-01 04:00:00,22871.0,11026.0,12198.0,0.707107,-7.071068e-01,0.006451,0.999979,1.0,529.0,14600.0,469.0
2022-01-01 10:00:00,4,2022-01-01 05:00:00,22423.0,10701.0,12195.0,0.500000,-8.660254e-01,0.007168,0.999974,1.0,527.0,14850.0,470.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 01:00:00,17515,2023-12-31 20:00:00,29329.7,30487.3,8204.0,0.258819,9.659258e-01,-0.007884,0.999969,0.0,1088.0,8596.0,373.0
2024-01-01 02:00:00,17516,2023-12-31 21:00:00,27283.2,30073.8,8204.0,0.500000,8.660254e-01,-0.007168,0.999974,0.0,909.0,8753.0,370.9
2024-01-01 03:00:00,17517,2023-12-31 22:00:00,26415.1,30143.0,8206.0,0.707107,7.071068e-01,-0.006451,0.999979,0.0,905.0,8635.0,366.0


In [130]:
trainData, valData, testData, fullTrainData = utility.splitDataset(dataset.values, NUM_TEST_DAYS, NUM_VAL_DAYS)
trainDates = dateTime[: -(NUM_TEST_DAYS*24)]
fullTrainDates = np.copy(trainDates)
trainDates, validationDates = trainDates[: -(NUM_VAL_DAYS*24)], trainDates[-(NUM_VAL_DAYS*24):]
testDates = dateTime[-(NUM_TEST_DAYS*24):]
trainData = trainData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]
valData = valData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]
testData = testData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]

No. of rows in dataset: 17520
No. of rows in training set: 12384
No. of rows in validation set: 720
No. of rows in test set: 4416


In [132]:
pd.DataFrame(trainData)

,0,1,2,3,4,5
0,12192.0,1.0,0.0,0.004301,0.999991,1.0
1,12192.0,0.965926,-0.258819,0.005017,0.999987,1.0
2,12192.0,0.866025,-0.5,0.005734,0.999984,1.0
3,12198.0,0.707107,-0.707107,0.006451,0.999979,1.0
4,12195.0,0.5,-0.866025,0.007168,0.999974,1.0
...,...,...,...,...,...,...
12379,10028.0,0.258819,0.965926,0.520649,-0.853771,0.0
12380,10034.0,0.5,0.866025,0.520037,-0.854144,0.0
12381,10038.0,0.707107,0.707107,0.519424,-0.854517,0.0
12382,10040.0,0.866025,0.5,0.518812,-0.854889,0.0


In [128]:
trainDates = dateTime[: -(NUM_TEST_DAYS*24)]

In [129]:
trainDates

array(['2022-01-01T06:00:00.000000000', '2022-01-01T07:00:00.000000000',
       '2022-01-01T08:00:00.000000000', ...,
       '2023-07-01T03:00:00.000000000', '2023-07-01T04:00:00.000000000',
       '2023-07-01T05:00:00.000000000'], dtype='datetime64[ns]')